# 1. Install necessary libraries

In [1]:
!pip install demucs
!pip install ipython

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 1.2/1.2 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: fin


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Import required modules

In [ ]:
import os
import sys
import subprocess
from pathlib import Path
from IPython.display import Audio, display

# 3. Define functions

In [ ]:
def separate_with_demucs(file_path: str, output_dir: Path):
    """Performs separation of the file using Demucs and saves the result in output_dir"""
    audio_path = Path(file_path)
    if not audio_path.exists():
        print(f"❌ File does not exist: {audio_path}")
        return

    output_dir.mkdir(parents=True, exist_ok=True)

    print(f"🎧 Running Demucs separation for file: {audio_path.name}\n")

    command = [
        sys.executable, "-m", "demucs",
        "-n", "htdemucs",
        str(audio_path)
    ]

    try:
        subprocess.run(command, check=True)

    except subprocess.CalledProcessError as e:
        print("❌ Error while running Demucs:", e)
        return

    # Looking for results in output_dir/htdemucs/<file_name> (default Demucs location)
    demucs_base = Path(output_dir) / "htdemucs" / audio_path.stem
    if not demucs_base.exists():
        print("❌ Demucs results not found:", demucs_base)
        return

    # Move all .wav files to our folder ./separated/<song_name>/
    for wav in demucs_base.glob("*.wav"):
        dest = output_dir / wav.name
        wav.replace(dest)
        print(f"✅ Saved: {dest}")

    # Remove the original Demucs folder (optional)
    try:
        for f in demucs_base.iterdir():
            f.unlink()
        demucs_base.rmdir()
    except Exception:
        pass

    print(f"\n🎵 Finished! All tracks saved in: {output_song_dir}\n")

# 4. Run separation on a sample file

In [ ]:
file_to_separate = "../musdb18/test/Al James - Schoolboy Facinatio.stem.mp4"
display(Audio(str(file_to_separate)))
output_dir = Path.cwd() / "separated" / Path(file_to_separate).stem

if Path(file_to_separate).exists():
    separate_with_demucs(file_to_separate, output_dir)
else:
    print(f"❌ Sample file does not exist: {file_to_separate}. Please provide a valid audio file path.")

# 5. Play separated tracks

In [ ]:
if 'output_directory' in locals() and output_dir is not None and output_dir.exists():
    for stem_file in sorted(output_dir.glob("*.wav")):
        print(f"--- {stem_file.name} ---")
        display(Audio(str(stem_file)))